In [1]:
from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy.linalg import solve
import numpy as np

# RWG1 Geometry calculations - all Chapters
Uses the structure mesh file, e.g. platefine.mat, as an input.

Creates the RWG edge element for every inner edge of the structure. The total number of elements is EdgesTotal.

Outputs the following arrays:

Edge first node number          

    Edge_(1,1:EdgesTotal)
   
Edge second node number  

    Edge_(2,1:EdgesTotal)
    
Plus triangle number  

    TrianglePlus(1:EdgesTotal)
   
Minus triangle number           

    TriangleMinus(1:EdgesTotal)
   
Edge length                     

    EdgeLength(1:EdgesTotal)
   
Edge element indicator          
    
    EdgeIndicator(1:EdgesTotal)

Also outputs areas and midpoints of separate triangles:

Triangle area                   

    Area(1:TrianglesTotal)
   
Triangle center                 

    Center(1:TrianglesTotal)      
   
This script may handle surfaces with T-junctions including monopoles over various metal surfaces and certain metal meshes

Copyright 2002 AEMM. Revision 2002/03/09 Chapter 2


In [2]:
mat_fname = pjoin('mesh', 'strip2.mat')

print(mat_fname)

mat_contents = sio.loadmat(mat_fname)

In [3]:
p = mat_contents['p']
t = mat_contents['t']

In [4]:
[s1,s2] = p.shape
if s1 == 2:
    print("Add third dimension")
    p = np.append(p,np.zeros((1,s2)),axis=0) 

In [5]:
TrianglesTotal=t.shape[1]
TrianglesTotal

Find areas of separate triangles

In [6]:
Area = np.zeros(TrianglesTotal)
Center = np.zeros((3,TrianglesTotal))
for m in range(TrianglesTotal):
    N = t[0:3,m] - 1
    #print(N)
    Vec1 = p[:,N[0]] - p[:,N[1]]
    Vec2 = p[:,N[2]] - p[:,N[1]]
    Area[m] = np.linalg.norm(np.cross(Vec1,Vec2))/2
    Center[:,m] = 1/3*sum(np.transpose(np.array(p[:,N])))

Find all edge elements "Edge_" with at least two adjacent triangles

In [7]:
n = 0
Edge_l = []
TrianglePlus = []
TriangleMinus = []
for m in range(TrianglesTotal):
    N = t[0:3,m] - 1 
    for k in range(m+1,TrianglesTotal):
        M = t[0:3,k] - 1      
        a = 1 - np.array([1 if i else 0 for i in [all(N-M[0]),all(N-M[1]),all(N-M[2])]])
        if(sum(a)==2): #triangles m and k have common edge
            Edge_l.append([M[i] for i,j in enumerate(a) if j]); 
            TrianglePlus.append(m);
            TriangleMinus.append(k); 
            n = n + 1;

In [8]:
Edge_ = np.transpose(np.array(Edge_l))
Edge__ = np.transpose(np.array([[i[1], i[0]] for i in Edge_l]))

EdgesTotal = len(Edge_l)
EdgesTotal

255

This block is only meaningful for T junctions. It leaves only two edge elements at a junction

In [9]:
Remove = []
for m in range(1,EdgesTotal):
    Edge_m = np.transpose(np.tile([np.array(Edge_)[:,m]],(EdgesTotal,1)))
    Edge_sup = list(Edge_  -Edge_m)
    Edge__sup = list(Edge__  -Edge_m)
    Ind1 = [0 if (Edge_sup[0][i] == 0) and (Edge_sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
    Ind2 = [0 if (Edge__sup[0][i] == 0) and (Edge__sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
    A_ = [i*j for i,j in zip(Ind1,Ind2)]
    A = [i for i,j in enumerate(A_) if j == 0]
    if(len(A)==3):    #three elements formally exist at a junction   
        print("There is a junction!! Stop!!")
        #Out=find(t(4,TrianglePlus(A))==t(4,TriangleMinus(A)));
        #Remove=[Remove A(Out)];

#np.array(list(Edge_).pop(Remove))
#TrianglePlus.pop(Remove)
#TriangleMinus.pop(Remove)
#EdgesTotal=len(Edge_)

EdgeIndicator = np.transpose(t[3,TrianglePlus] + t[3,TriangleMinus])

Find edges' length

In [10]:
EdgeLength = np.zeros(EdgesTotal)
for m in range(EdgesTotal):
    EdgeLength[m]=np.linalg.norm(p[:,Edge_[0,m]]-p[:,Edge_[1,m]])

In [11]:
Center.shape

(3, 244)

# RWG2: Geometry calculations - all Chapters
Uses the mesh file from RWG1, mesh1.mat, as an input.

Creates the following parameters of the RWG edge elements: 

Position vector rho_c_plus from the free vertex of the "plus" triangle to its center

                                   RHO_Plus(1:3,1:EdgesTotal)
                                   
Position vector rho_c_minus from the center of the "minus" triangle to its free vertex 

                                   RHO_Minus(1:3,1:EdgesTotal)

In addition to these parameters creates the following arrays for nine subtriangles (barycentric subdivision):

Midpoints of nine subtriangles

                                   Center_(1:3,1:9,1:TrianglesTotal)  
                                   
Position vectors rho_c_plus from the free vertex of the "plus" triangle to nine subtriangle midpoints

                                   RHO__Plus(1:3,1:9,1:EdgesTotal)
                                   
Position vectors rho_c_minus from nine subtriangle midpoints to the free vertex of the "minus" triangle

                                   RHO__Minus(1:3,1:9,1:EdgesTotal)

See Rao, Wilton, Glisson, IEEE Trans. Antennas and Propagation, vol. AP-30, No 3, pp. 409-418, 1982.

Copyright 2002 AEMM. Revision 2002/03/05 
Chapter 2

In [12]:
IMT=[]
for m in range(TrianglesTotal):
    n1 = t[0,m] - 1
    n2 = t[1,m] - 1
    n3 = t[2,m] - 1
    M = Center[:,m]
    r1 = p[:,n1]
    r2 = p[:,n2]
    r3 = p[:,n3]
    r12 = r2-r1
    r23 = r3-r2
    r13 = r3-r1
    C1 = r1 + (1/3)*r12
    C2 = r1+(2/3)*r12
    C3 = r2+(1/3)*r23
    C4 = r2+(2/3)*r23
    C5 = r1+(1/3)*r13
    C6 = r1+(2/3)*r13
    a1 = np.transpose([1/3*(C1+C5+r1)])
    a2 = np.transpose([1/3*(C1+C2+M)])
    a3 = np.transpose([1/3*(C2+C3+r2)])
    a4 = np.transpose([1/3*(C2+C3+M)])
    a5 = np.transpose([1/3*(C3+C4+M)])
    a6 = np.transpose([1/3*(C1+C5+M)])
    a7 = np.transpose([1/3*(C5+C6+M)])
    a8 = np.transpose([1/3*(C4+C6+M)])
    a9 = np.transpose([1/3*(C4+C6+r3)])
    if m == 0:
        Center_ = [np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9), axis=1)]
    else:
        Center_ = np.concatenate((Center_,[np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9), axis=1)]), axis = 0)
        
Center_ = Center_.transpose(1,2,0)


In [13]:
Center_.shape

(3, 9, 244)

In [14]:
Center_[:,:,0]

array([[ 0.00666667,  0.        , -0.00666667, -0.00333333, -0.00333333,
         0.00333333,  0.00333333,  0.        ,  0.        ],
       [ 0.99927397,  0.99927397,  0.99927397,  0.99854794,  0.99709588,
         0.99854794,  0.99709588,  0.99636985,  0.9949178 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

### PLUS

In [15]:
for m in range(EdgesTotal):
    NoPlus=TrianglePlus[m]
    n1 = t[0,NoPlus] - 1
    n2 = t[1,NoPlus] - 1
    n3 = t[2,NoPlus] - 1
    if((n1 != Edge_[0,m])&(n1 != Edge_[1,m])): 
        NODE=n1
    if((n2 != Edge_[0,m])&(n2 != Edge_[1,m])): 
        NODE=n2
    if((n3 != Edge_[0,m])&(n3 != Edge_[1,m])): 
        NODE=n3
    FreeVertex=p[:,NODE]
    
    if m == 0:
        RHO_Plus   = [Center[:,NoPlus]-FreeVertex]
        #Nine rho's of the "plus" triangle
        RHO__Plus  = [Center_[:,:,NoPlus] - np.transpose(np.tile([FreeVertex],(9,1)))]
    else:
        RHO_Plus   = np.concatenate((RHO_Plus,[Center[:,NoPlus]-FreeVertex]), axis = 0)
        RHO__Plus  = np.concatenate((RHO__Plus,[Center_[:,:,NoPlus] - np.transpose(np.tile([FreeVertex],(9,1)))]), axis = 0)

RHO_Plus = RHO_Plus.transpose(1,0)
RHO__Plus = RHO__Plus.transpose(1,2,0)

In [16]:
RHO_Plus.shape

(3, 255)

In [17]:
RHO__Plus[:,:,0]

array([[ 0.01666667,  0.01      ,  0.00333333,  0.00666667,  0.00666667,
         0.01333333,  0.01333333,  0.01      ,  0.01      ],
       [-0.00072603, -0.00072603, -0.00072603, -0.00145206, -0.00290412,
        -0.00145206, -0.00290412, -0.00363015, -0.0050822 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [18]:
RHO__Plus.shape

(3, 9, 255)

In [19]:
m = 0
NoPlus=TrianglePlus[m]
n1 = t[0,NoPlus] - 1
n2 = t[1,NoPlus] - 1
n3 = t[2,NoPlus] - 1
if((n1 != Edge_[0,m])&(n1 != Edge_[1,m])): 
    NODE=n1
if((n2 != Edge_[0,m])&(n2 != Edge_[1,m])): 
    NODE=n2
if((n3 != Edge_[0,m])&(n3 != Edge_[1,m])): 
    NODE=n3
FreeVertex=p[:,NODE]

### MINUS

In [20]:
for m in range(EdgesTotal):
    NoMinus=TriangleMinus[m]
    n1 = t[0,NoMinus] - 1
    n2 = t[1,NoMinus] - 1
    n3 = t[2,NoMinus] - 1
    if((n1 != Edge_[0,m])&(n1 != Edge_[1,m])): 
        NODE=n1
    if((n2 != Edge_[0,m])&(n2 != Edge_[1,m])): 
        NODE=n2
    if((n3 != Edge_[0,m])&(n3 != Edge_[1,m])): 
        NODE=n3
    FreeVertex=p[:,NODE]
    
    if m == 0:
        RHO_Minus   = [-Center[:,NoMinus] + FreeVertex]
        #Nine rho's of the "plus" triangle
        RHO__Minus  = [-Center_[:,:,NoMinus] + np.transpose(np.tile([FreeVertex],(9,1)))]
    else:
        RHO_Minus   = np.concatenate((RHO_Minus,[-Center[:,NoMinus] + FreeVertex]), axis = 0)
        RHO__Minus  = np.concatenate((RHO__Minus,[-Center_[:,:,NoMinus] + np.transpose(np.tile([FreeVertex],(9,1)))]), axis = 0)

RHO_Minus = RHO_Minus.transpose(1,0)
RHO__Minus = RHO__Minus.transpose(1,2,0)

In [21]:
RHO_Minus.shape

(3, 255)

In [22]:
RHO__Minus.shape

(3, 9, 255)

In [23]:
RHO_Minus.shape

(3, 255)

# RWG3 Calculates the impedance matrix using function IMPMET
   Uses the mesh file from RWG2, mesh2.mat, as an input.

The following parameters need to be specified prior to calculations:

Frequency (Hz)                  f

Dielectric constant (SI)        epsilon_

Magnetic permeability (SI)      mu_

Copyright 2002 AEMM. Revision 2002/03/11 
Chapter 2

EM parameters (f=3e8 means that f=300 MHz) 

In [24]:
f           = 75e6 
epsilon_    = 8.854e-012
mu_         = 1.257e-006

Speed of light

In [25]:
c_=1/np.sqrt(epsilon_*mu_)

Free-space impedance

In [26]:
eta_=np.sqrt(mu_/epsilon_)

Contemporary variables - introduced to speed up the impedance matrix calculation

In [27]:
omega       =2*np.pi*f                                        
k           =omega/c_
K           =1j*k
Constant1   =mu_/(4*np.pi)
Constant2   =1/(1j*4*np.pi*omega*epsilon_)
Factor      =1/9

In [28]:
FactorA     =Factor*(1j*omega*EdgeLength/4)*Constant1
FactorFi    =Factor*EdgeLength*Constant2

In [29]:
for m in range(EdgesTotal):
    if m == 0:
        RHO_P=[np.transpose(np.tile([RHO_Plus[:,m]],(9,1)))]   #[3 9 EdgesTotal]
        RHO_M=[np.transpose(np.tile([RHO_Minus[:,m]],(9,1)))]  #[3 9 EdgesTotal]
    else:
        RHO_P=np.concatenate((RHO_P,[np.transpose(np.tile([RHO_Plus[:,m]],(9,1)))]), axis = 0)   #[3 9 EdgesTotal]
        RHO_M=np.concatenate((RHO_M,[np.transpose(np.tile([RHO_Minus[:,m]],(9,1)))]), axis = 0)  #[3 9 EdgesTotal]
        
RHO_P = RHO_P.transpose(1,2,0)
RHO_M = RHO_M.transpose(1,2,0)

In [30]:
RHO_M.shape

(3, 9, 255)

In [31]:
RHO_M[:,:,3]

array([[-0.00333333, -0.00333333, -0.00333333, -0.00333333, -0.00333333,
        -0.00333333, -0.00333333, -0.00333333, -0.00333333],
       [-0.00367173, -0.00367173, -0.00367173, -0.00367173, -0.00367173,
        -0.00367173, -0.00367173, -0.00367173, -0.00367173],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [32]:
FactorA = FactorA[np.newaxis].T
FactorFi = FactorFi[np.newaxis].T

## Impedance matrix Z
### IMPMET Standard impedance matrix (metal surface)

Returns the complex impedance matrix 

    [EdgesTotal x EdgesTotal]
    
Uses 9 integration points for every triangle 
(barycentric subdivision)

The impedance matrix is calculated as a sum of the contributions due to separate triangles (similar to the "face-pair" method). 
See Appendix B for a detailed algorithm.
 
A 9-point quadrature is used for all integrals, including the self-coupling terms. The alternative source code with the analytical approximation of the self-coupling terms is given in Appendix B. The difference between two methods is not significant. 

Copyright 2002 AEMM. Revision 2002/03/12 
Chapter 2

Memory allocation

In [33]:
Z   = np.zeros ((EdgesTotal,EdgesTotal))+1j*np.zeros((EdgesTotal,EdgesTotal))

In [34]:
Z.shape

(255, 255)

Loop over integration triangles

In [35]:
for p_ in range(TrianglesTotal):
    Plus  = [i for i,j in enumerate(np.array(TrianglePlus)-p_) if j == 0]
    Minus  = [i for i,j in enumerate(np.array(TriangleMinus)-p_) if j == 0]

    D = Center_ - np.transpose(np.tile([Center[:,p_]],(TrianglesTotal,9,1))) #[3 9 TrianglesTotal]     
    
    R=np.sqrt(sum(D*D))[np.newaxis]                              #[1 9 TrianglesTotal]
    g=np.exp(-K*R)/R                                #[1 9 TrianglesTotal]
    
    gP=g[:,:,TrianglePlus]                         #[1 9 EdgesTotal]
    gM=g[:,:,TriangleMinus]                        #[1 9 EdgesTotal]
    
    Fi= (np.sum(gP,axis = 1) - np.sum(gM,axis = 1))#[1 1 EdgesTotal]
    ZF= FactorFi*Fi.T         #[EdgesTotal 1]
    
    for n in Plus:
        #n = Plus[k]
        RP = np.tile([RHO__Plus[:,:,n]],(EdgesTotal,1,1)).transpose(1,2,0)  #[3 9 EdgesTotal]
        A=(np.sum(gP*np.sum(RP*RHO_P,axis=0),axis=1)+np.sum(gM*np.sum(RP*RHO_M,axis=0),axis=1))
        Z1= FactorA*A.T
        Z[:,n]=Z[:,n] + (EdgeLength[n]*(Z1+ZF)).reshape(EdgesTotal)
        
    for n in Minus:
        #n = Minus[k]
        RP = np.tile([RHO__Minus[:,:,n]],(EdgesTotal,1,1)).transpose(1,2,0)  #[3 9 EdgesTotal]
        A=(np.sum(gP*np.sum(RP*RHO_P,axis=0),axis=1)+np.sum(gM*np.sum(RP*RHO_M,axis=0),axis=1))
        Z1= FactorA*A.T
        Z[:,n]=Z[:,n] + (EdgeLength[n]*(Z1-ZF)).reshape(EdgesTotal)

In [36]:
print('Is Z[1,186] (-2.0246662922063444e-07-2.919507910419309e-07j)?')
Z[1,186]==(-2.0246662922063444e-07-2.919507910419309e-07j)

Is Z[1,186] (-2.0246662922063444e-07-2.919507910419309e-07j)?


True

# RWG4 Solves MoM equations for the antenna radiation problem
Uses the mesh file from RWG2, mesh2.mat, and the impedance file from RWG3, impedance.mat, as inputs.

Also calculates the "voltage" vector V (the right-hand side of moment equations)         
        
        V(1:EdgesTotal)

The following parameters need to be specified:

The feed point position                 

        FeedPoint(1:3);

Number of feeding edges (one for the dipole; two for the monopole)

        INDEX(1:2)

Copyright 2002 AEMM. Revision 2002/03/14 
Chapter 4

### Find the feeding edge(s)(closest to the origin)

In [37]:
FeedPoint=[[0],[0],[0]]

In [38]:
V = np.zeros (EdgesTotal)

In [39]:
Distance = np.zeros((3,EdgesTotal))
for m in range(EdgesTotal):
    Distance[:,m]=(0.5*np.sum(p[:,Edge_[:,m]],axis=1)[np.newaxis].T-FeedPoint).reshape(3)

In [40]:
np.sum(Distance*Distance,axis=0).shape

(255,)

In [41]:
INDEX=np.argsort(np.sum(Distance*Distance,axis=0),kind='mergesort') 
Index = INDEX[0]               #Center feed - dipole
#Index=INDEX[0:1]              #Probe feed - monopole

In [42]:
print('Is Index 114?')
Index==114

Is Index 114?


True

### Define the voltage vector

In [43]:
V[Index]=1*EdgeLength[Index]

### Solve system of MoM equations

In [44]:
I=solve(Z, V)

In [45]:
print('Is I[200] (0.12188196388339129-0.09287114648145402j)?')
I[200]==(0.12188196388339129-0.09287114648145402j)

Is I[200] (0.12188196388339129-0.09287114648145402j)?


True

### Find the antenna input impedance

In [46]:
GapCurrent  =np.sum(I[Index]*EdgeLength[Index].T)
GapVoltage  =np.mean(V[Index]/EdgeLength[Index])
Impedance   =GapVoltage/GapCurrent
FeedPower   =1/2*np.real(GapCurrent*np.conj(GapVoltage))

In [47]:
print('The impedance of the given antenna is: ', Impedance)

The impedance of the given antenna is:  (87.57081374415527+47.29383785024352j)
